In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from accelerate import load_checkpoint_and_dispatch, Accelerator

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("../autodl-fs/llama7B")

In [3]:
# tokenizer.padding_side='left'

In [4]:
prompt = ["Hey, are you consciours? Can you talk to me?", "Hello!"]
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(prompt, return_tensors="pt", 
                                padding=True,
                                truncation='longest_first',
                                max_length=512)

In [5]:
inputs

{'input_ids': tensor([[    1, 18637, 29892,   526,   366,  1136,   455,  2470, 29973,  1815,
           366,  5193,   304,   592, 29973],
        [    1, 15043, 29991,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [6]:
model = LlamaForCausalLM.from_pretrained("../autodl-fs/llama7B")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
outputs = model(**inputs, labels=inputs["input_ids"])

In [8]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values'])

In [10]:
outputs['logits'].size()

torch.Size([2, 15, 32000])

In [16]:
sample_output = model.generate(
    **inputs, 
    do_sample=True, 
    max_length=50, 
    top_p=0.95, 
    top_k=50,
    pad_token_id=tokenizer.eos_token_id,
    early_stopping=True,
    num_return_sequences=3
)

In [17]:
for i in range(6):
    print(tokenizer.decode(sample_output[i], skip_special_tokens=False))

<s>Hey, are you consciours? Can you talk to me?
Yeah, I’m on Instagram, I’m in the fashion industry, and I like photography.
What about the fashion industry?
I
<s>Hey, are you consciours? Can you talk to me?
I'm doing a blog on fashion, I will talk about fashion, music and beauty, but I'm doing a special project! Do you want to talk and
<s> Hey, are you consciours? Can you talk to me? If you do, then please contact with me, I will tell you my story.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>
</s></s></s></s></s></s></s></s></s></s></s></s><s>Hello! I am so excited to be a part of the Keller Williams family in such a new way! As a native of the Charleston area I have had the privilege
</s></s></s></s></s></s></s></s></s></s></s></s><s>Hello! My name is Jill McDonald and I teach 3rd grade at Cromwell Elementary. I have been teaching 3rd grade since 2009
</s></s></s></s></s></s></s></s></s></s></s></s><s>Hello! First post, and I just wanted to say hello! So, I

In [3]:
gpu_model = load_checkpoint_and_dispatch(
    LlamaForCausalLM, "../autodl-fs/llama13B", device_map="auto", no_split_module_classes=["GPTJBlock"]
)

TypeError: named_parameters() missing 1 required positional argument: 'self'